# 01.2 - Kedro framework

<img src="../static/kedro-horizontal-color-on-light.png" width="400" alt="Kedro">

This notebook covers how to use the Kedro CLI to create and manage projects using the Kedro framework, which assembles the library components seen in [First steps with Kedro](./01_1-First%20Steps%20with%20Kedro.ipynb) in a standard way.

## Starters

A Kedro starter contains code in the form of a Cookiecutter template for a Kedro project. Using a starter is like using a pre-defined layout when creating a presentation or document.

You can find [the official list of starters](https://docs.kedro.org/en/0.19.10/starters/starters.html#official-kedro-starters) in the documentation.

The basis for the rest of this notebook will be the [`spaceflights-pandas`](https://github.com/kedro-org/kedro-starters/tree/0.19.10/spaceflights-pandas) starter, which is ideal for local execution. Later on in the bootcamp you will move on to the `spaceflights-pyspark` starter, which uses PySpark and is ready to be executed in Databricks.

To use it, you will first need `kedro` installed. You can use `conda`, a separate virtual environment, or a Python workflow tool capable of managing global utilities, such as pipx or uv.

```bash
(.venv) $ kedro new --starter=spaceflights-pandas --name rocketfuel
```


## The directory structure

A typical Kedro project looks like this:

```
project-dir          # Parent directory of the template
├── conf             # Project configuration files
├── data             # Local project data (not committed to version control)
├── docs             # Project documentation
├── notebooks        # Project-related Jupyter notebooks (can be used for experimental code before moving the code to src)
├── src              # Project source code
├── tests            # Folder containing unit and integration tests
├── .gitignore       # Hidden file that prevents staging of unnecessary files to `git`
├── pyproject.toml   # Identifies the project root and contains configuration information
├── README.md        # Project README
├── requirements.txt # Project dependencies file
```


## Running pipelines

You can use the Kedro CLI to run any pipeline that has been registered.

To see the pipelines already registered, in your project, run

```bash
(.venv) $ kedro registry list
```

Now you can use `kedro run` to execute the pipeline of your liking:

```bash
(.venv) $ kedro run --pipeline data_processing
```

And after the `data_processing` pipeline is executed, you should be able to see the results in the appropriate local directory:

In [ ]:
%%sh
cd rocketfuel && ls data/02_intermediate


## Environments

A [configuration environment](https://docs.kedro.org/en/0.19.10/configuration/configuration_basics.html#configuration-environments) is a way of organising your configuration settings for different stages of your data pipeline. For example, you might have different settings for development, testing, and production environments.

By default, Kedro projects have a `base` and a `local` environment.


## Pipeline creation

To create a new pipeline, you can use the CLI:

```bash
(.venv) $ kedro pipeline create PIPELINE_NAME
```


## Integration with VS Code

Kedro has an official extension for VS Code, providing features like enhanced code navigation and autocompletion for seamless development.


## Visualisation with Kedro Viz

You can use Kedro Viz to visualise your pipelines in 3 ways:

1. Using the `NotebookVisualizer` (see 01-1 notebook)
2. Using the VS Code integration https://docs.kedro.org/en/stable/development/set_up_vscode.html#visualise-the-pipeline-with-kedro-viz
3. Launching the Kedro Viz web application on the command line:

```bash
(.venv) $ kedro viz run
```

### Exercise 5

Register two new pipelines in the pipeline registry called `train` and `inference` that use the appropriate nodes from the `data_science` pipeline.

Visualise them in Kedro Viz.


## Remote data paths in the catalog

Kedro datasets can be virtually anything: database connections, REST APIs, and of course files.

These files can be referenced by using remote filepaths, not just local ones.

Most [official Kedro datasets](https://docs.kedro.org/projects/kedro-datasets/en/kedro-datasets-7.0.0/api/kedro_datasets.html) use [`fsspec`](https://filesystem-spec.readthedocs.io/),
a Python library that allows users to easily specify remote filepaths
with [lots of different cloud filesystems](https://filesystem-spec.readthedocs.io/en/latest/api.html#built-in-implementations),
including DBFS.

Consider this dataset definition:

```
companies:
  type: spark.SparkDataset
  filepath: dbfs:/Volumes/aza-databricks-b9b7aae-catalog/rocketfuel/bronze/companies.csv
  file_format: csv
```

Provided that `databricks-connect` is properly configured and the data available in the right volume (see instructions for Day 02), this will trivially work:

In [ ]:
%pip install hdfs s3fs

In [ ]:
from kedro.io import DataCatalog

catalog_remote_spark = DataCatalog.from_config({
    "companies": {
        "type": "spark.SparkDataset",
        "filepath": "dbfs:/Volumes/aza-databricks-b9b7aae-catalog/rocketfuel/bronze/companies.csv",
        "file_format": "csv",
    }
})

In [ ]:
catalog_remote_spark.load("companies").show(5)

_**Note**: This example uses native PySpark reading capabilities. fsspec uses the DBFS API, which is not compatible with Unity Catalog Volumes https://github.com/fsspec/filesystem_spec/issues/1656_